In [1]:
import json
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

In [2]:
def load_data(path, closest_d_tol=1e-6):

    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    exact_min = np.min(data['eigenvalues'])
    final_k = data['final_k']

    all_run_data = data["all_run_data"]
    keys = all_run_data[0].keys()

    cols = {k: [d.get(k) for d in all_run_data] for k in keys}

    reduction_percentages = cols["reduction"]
    reduced_eigenvalues = cols["H_reduced_e"]
    diff_to_mins = cols["diff"]

    first_idx_tol = next((i for i, d in enumerate(diff_to_mins) if abs(d) <= closest_d_tol), None) #index of first step that is within specified tolerance

    num_samples = cols["num_samples"]

    row = {
        "N": data["N"],
        "cutoff": data["cutoff"],
        "exact_min": exact_min,
        "final_k": final_k,
        "reduced_eigenvalues": reduced_eigenvalues,
        "reduction_percentages": reduction_percentages,
        "num_samples": num_samples,
        "diff_to_mins": diff_to_mins,
        "first_idx_tol": first_idx_tol
    }

    return row

In [3]:
def plot_individual(data, show=True, save=False):

    potential = data['potential']
    cutoff = data['cutoff']
    idx_tol =data['first_idx_tol']

    y_left = data['reduction_percentages']
    y_right = data["diff_to_mins"]
    x = range(data['final_k'])

    fig, ax1 = plt.subplots(figsize=(9, 5))


    # Left axis
    l1 = ax1.plot(x, y_left, marker="o", label="H Reduction %")
    ax1.set_xlabel('D')
    ax1.set_ylabel("H Reduction %")
    ax1.grid(True, alpha=0.3)

    # Right axis
    ax2 = ax1.twinx()
    l2 = ax2.plot(x, y_right, marker="s", color='r', label="Diff to Min")
    ax2.set_ylabel("Diff to Min")

    # if idx_tol: 
    #     vl = ax1.axvline(x=idx_tol, linestyle="--", color="k", linewidth=1.5, alpha=0.8, zorder=0, label=f"threshold = {data['conv_tol']}")
    #     lines = l1 + l2 + [vl]
    # else:
    lines = l1 + l2
        
    labels = [ln.get_label() for ln in lines]
    ax1.legend(lines, labels, loc="best")

    plt.title(f"{data['label']} - {data['bc']} - {potential} - N{data['N']} - $\\Lambda${cutoff}")
    plt.tight_layout()

    if show and not save:
        plt.show()
    
    if save:
        save_path = str(data['path']).replace(f'{potential}_{cutoff}.json', '')
        os.makedirs(save_path, exist_ok=True)
        plt.savefig(os.path.join(save_path, f"{potential}_{cutoff}.png"))
        plt.close()


In [7]:
#root = os.path.join(repo_path, r"SUSY\Wess-Zumino\Qiskit\SBQKD\AnsatzComp")
#labels = ['EvolOnly','BasisOnly','Basis+RYs','Basis+XXYYs','Full']

root = os.path.join(repo_path, r"SUSY\Wess-Zumino\Qiskit\SBQKD\TopN-L2")
labels = os.listdir(root)


sites = [2,3,4,5,6]
cutoffs = [2,4,8,16]
#sites=[5]
bc = "dirichlet"
potential = "linear"

all_data = []
conv_tol = 1e-6
for label in labels:
    for site in sites:
        for cutoff in cutoffs:

            path = os.path.join(root, label, bc, potential, f"N{site}", f"{potential}_{cutoff}.json")
            try:
                data = load_data(path, conv_tol)
            except:
                continue
            
            data['path'] = path
            data['label'] = label
            data['potential'] = potential
            data['bc'] = bc
            data['conv_tol'] = conv_tol

            plot_individual(data, show=False, save=True)

            all_data.append(data)

In [8]:
def plot_multi(
    data,
    ax=None,
    show=True,
    is_perc=True,
    add_legend=True,
    mark_convergence=True,
    conv_registry=None,
    conv_jitter_frac=0.006,
    conv_marker="x",
    colour='b'
):
    if conv_registry is None:
        conv_registry = {}

    created = False
    if ax is None:
        fig, ax = plt.subplots(figsize=(9, 5))
        created = True

    x = np.arange(int(data["final_k"]))
    if is_perc:
        y=(np.asarray([100.00]*len(data["reduction_percentages"]))-np.asarray((data["reduction_percentages"])))
    else:
        y = np.asarray(data["diff_to_mins"], dtype=float)

    # main line
    (line,) = ax.plot(
        x, y,
        marker="o", markersize=4, linewidth=1.5, color=colour,
        label=data.get("label", "")
    )

    ax.grid(True, alpha=0.3)

    # convergence handling
    idx_tol = data.get("first_idx_tol", None)
    if idx_tol is not None:
        idx_tol = int(idx_tol)
        idx_tol = max(0, min(idx_tol, len(y) - 1))

        if mark_convergence:
            x_span = float(np.max(x) - np.min(x)) if len(x) > 1 else 1.0
            jitter = conv_jitter_frac * x_span

            used = conv_registry.get(idx_tol, 0)
            conv_registry[idx_tol] = used + 1

            if used == 0:
                dx = 0.0
            else:
                k = (used + 1) // 2
                dx = (k * jitter) * (1 if used % 2 == 1 else -1)

            ax.plot(
                [idx_tol + dx],
                [y[idx_tol]],
                marker=conv_marker,
                linestyle="None",
                markersize=6,
                markeredgecolor=line.get_color(),
                markeredgewidth=1.0,
                zorder=0,
            )

    if add_legend:
        ax.legend(loc="best", fontsize=9)

    if created:
        plt.tight_layout()
        if show:
            plt.show()

    return ax, conv_registry


In [10]:
bc = "dirichlet"
potential = "linear"
colours = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"]
#out_dir = os.path.join(repo_path, r"SUSY\Wess-Zumino\Qiskit\SBQKD\TopN-L2\Plots\EDiff")
out_dir = os.path.join(repo_path, r"SUSY\Wess-Zumino\Qiskit\SBQKD\TopN-L2\Plots\PercentageReduction")
os.makedirs(out_dir, exist_ok=True)

is_perc=True

for N in sites:
    for cutoff in cutoffs:
        data = [d for d in all_data if (d.get("N") == N) and (d.get("cutoff") == cutoff)]
        if not data:
            continue

        fig, ax = plt.subplots(figsize=(9, 5))
        registry = {}

        # Set once per figure
        ax.set_xlabel("D")
        if is_perc:
            ax.set_ylabel("H Reduction %")
        else:
            ax.set_ylabel("|$\\Delta_{min}$|")
        ax.grid(True, alpha=0.3)

        for i, d in enumerate(data):
            
            colour = colours[i % len(colours)]
            ax, registry = plot_multi(
                d,
                ax=ax,
                show=False,
                is_perc=is_perc,
                add_legend=False,
                mark_convergence=True,
                conv_registry=registry,
                colour=colour
            )

        ax.legend(loc="best", fontsize=9)
        ax.set_title(f"{bc} - {potential} - N{N} - $\\Lambda${cutoff}")
        if not is_perc: plt.yscale('log')
        plt.tight_layout()


        save_path = os.path.join(out_dir, f"{bc}-{potential}-N{N}-L{cutoff}.png")
        plt.savefig(save_path, dpi=200)
        plt.close(fig)
        #plt.show()



In [34]:
def load_data(path, closest_d_tol=1e-6):

    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    exact_min = data['min_eigenvalue']

    all_run_data = data["all_run_data"]
    keys = all_run_data[0].keys()

    cols = {k: [d.get(k) for d in all_run_data] for k in keys}

    num_2q_ops = [
    (d.get("circuit_cost") or {}).get("num_2q_ops", d.get("num_2q_ops"))
    for d in all_run_data
    ]
    depth = [
        (d.get("circuit_cost") or {}).get("depth", d.get("depth"))
        for d in all_run_data
    ]

    reduction_percentages = cols["reduction"]
    reduced_eigenvalues = cols["H_reduced_e"]
    diff_to_mins = cols["diff_to_exact"]
    final_k = max(cols['D'])
    t = cols['t']
    dt = cols['dt']
    dt_slice_max = cols['dt_slice_max']
    reps = cols['reps']
    synthesis = cols['synthesis']

    first_idx_tol = next((i for i, d in enumerate(diff_to_mins) if abs(d) <= closest_d_tol), None) #index of first step that is within specified tolerance

    num_samples = cols["num_samples"]

    row = {
        "N": data["N"],
        "cutoff": data["cutoff"],
        "t": t,
        "dt": dt[0],
        "dt_slice_max": dt_slice_max[0],
        "reps": reps,
        "synthesis":synthesis[0],
        "final_k": final_k,
        "exact_min": exact_min,
        "reduced_eigenvalues": reduced_eigenvalues,
        "reduction_percentages": reduction_percentages,
        "final_reduction": reduction_percentages[-1],
        "reduction_at_first_idx": reduction_percentages[first_idx_tol],
        #"num_samples": num_samples,
        "diff_to_mins": diff_to_mins,
        "best_diff_to_min": min(diff_to_mins),
        "first_idx_tol": first_idx_tol,
        "num_2q_ops": num_2q_ops,
        "max_num_2q_ops": max(x for x in num_2q_ops if x is not None),
        "num_2q_ops_at_first_idx": num_2q_ops[first_idx_tol],
        "depth": depth,
        "max_depth": max(x for x in depth if x is not None),
        "depth_at_first_idx": depth[first_idx_tol],
    }

    return row

In [35]:
root = os.path.join(repo_path, r"SUSY\Wess-Zumino\Qiskit\SBQKD\Files\Optimize\dirichlet\linear")

bc = "dirichlet"
potential = "linear"
sites = [2,3,4,5,6]
cutoffs = [2,4,8,16]


conv_tol = 1e-6

colours = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"]
is_perc=True

top_5s = []

for N in sites:
    f_path = os.path.join(root, f"N{N}", "sweeps")
    folders = [name for name in os.listdir(f_path) if os.path.isdir(os.path.join(f_path, name))]

    for cutoff in cutoffs:

        print(N, cutoff)

        for folder in folders:
        
            d_path = os.path.join(f_path, folder, f"{potential}_{cutoff}.json")
            try:
                data = load_data(d_path, conv_tol)
            except:
                continue

            #data['path'] = root
            #data['potential'] = potential
            #data['bc'] = bc
            #data['conv_tol'] = conv_tol

            all_data.append(data)
            df = pd.DataFrame(data)
            cols = ['N', 'cutoff', 'dt', 'dt_slice_max', 'synthesis', 'final_k', 'final_reduction', 'reduction_at_first_idx', 'best_diff_to_min', 'first_idx_tol', 'max_num_2q_ops', 'num_2q_ops_at_first_idx']
            red = df[cols]
            sorted = red.sort_values('best_diff_to_min').head(5)#.values
            top_5s.append(sorted)

        break

    break

2 2


In [38]:
root = os.path.join(repo_path, r"SUSY\Wess-Zumino\Qiskit\SBQKD\Files\Optimize\dirichlet\linear")

bc = "dirichlet"
potential = "linear"
sites = [2,3,4,5,6]
cutoffs = [2,4,8,16]


conv_tol = 1e-6

colours = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"]
is_perc=True

top_5s = []
all_data = []

for N in sites:
    f_path = os.path.join(root, f"N{N}", "sweeps")
    folders = [name for name in os.listdir(f_path) if os.path.isdir(os.path.join(f_path, name))]

    for cutoff in cutoffs:
        rows = []  # collect all rows across folders for this (N, cutoff)

        for folder in folders:
            d_path = os.path.join(f_path, folder, f"{potential}_{cutoff}.json")
            try:
                data = load_data(d_path, conv_tol)
            except Exception:
                continue

            df = pd.DataFrame([data])
            cols = [
                "N", "cutoff", "dt", "dt_slice_max", "synthesis", "final_k",
                'final_reduction', 'reduction_at_first_idx', "best_diff_to_min", "first_idx_tol",
                "max_num_2q_ops", "num_2q_ops_at_first_idx",
            ]
            red = df[cols].copy()
            red["folder"] = folder  # optional, but super helpful for tracing
            rows.append(red)

        if not rows:
            continue  # nothing found for this (N, cutoff)

        all_red = pd.concat(rows, ignore_index=True)

        top5 = all_red.nsmallest(5, "best_diff_to_min").reset_index(drop=True)

        # optional: store identifiers so you can keep track when concatenated later
        top5["N"] = N
        top5["cutoff"] = cutoff

        top_5s.append(top5)
        all_data.append(all_red)

# If you want a single dataframe with a MultiIndex (N, cutoff):
top5_df = pd.concat(top_5s, ignore_index=True)
# or: top5_df = pd.concat(top_5s, keys=[(df["N"].iloc[0], df["cutoff"].iloc[0]) for df in top_5s])


In [42]:
df_runs = pd.concat(all_data, ignore_index=True)

In [48]:
def pareto_front(df, minimize, maximize):
    X = df.copy()

    # Convert all to "minimize" by negating maximize metrics
    cols = []
    for c in minimize:
        cols.append(c)
    for c in maximize:
        X[c] = -X[c]
        cols.append(c)

    vals = X[cols].to_numpy()

    is_pareto = np.ones(len(X), dtype=bool)
    for i in range(len(X)):
        if not is_pareto[i]:
            continue
        # j dominates i if j <= i for all metrics and < for at least one
        dominates = np.all(vals <= vals[i], axis=1) & np.any(vals < vals[i], axis=1)
        is_pareto[dominates] = True
        # i is dominated by someone else?
        if np.any(np.all(vals <= vals[i], axis=1) & np.any(vals < vals[i], axis=1)):
            # keep i if it isn't dominated (we check domination by others)
            pass

    # Proper domination test:
    is_pareto = np.ones(len(X), dtype=bool)
    for i in range(len(X)):
        for j in range(len(X)):
            if i == j:
                continue
            if np.all(vals[j] <= vals[i]) and np.any(vals[j] < vals[i]):
                is_pareto[i] = False
                break

    return df[is_pareto].copy()


minimize = ["final_k", "best_diff_to_min", "first_idx_tol", "max_num_2q_ops", "num_2q_ops_at_first_idx",]
maximize = ['final_reduction', 'reduction_at_first_idx']

pf = pareto_front(df_runs, minimize=minimize, maximize=maximize)
pf#.sort_values("best_diff_to_min").head(20)


,N,cutoff,dt,dt_slice_max,synthesis,final_k,final_reduction,reduction_at_first_idx,best_diff_to_min,first_idx_tol,max_num_2q_ops,num_2q_ops_at_first_idx,folder
6,2,2,1.00,0.125,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.125
7,2,2,1.00,0.250,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.25
8,2,2,1.00,0.500,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.5
9,2,2,1.50,0.125,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.5_dtslice=0.125
10,2,2,1.50,0.250,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.5_dtslice=0.25
21,2,2,0.50,0.125,suzuki2,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=suzuki2_dt=0.5_dtslice=0.125
22,2,2,0.50,0.250,suzuki2,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=suzuki2_dt=0.5_dtslice=0.25
27,2,2,1.50,0.125,suzuki2,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=suzuki2_dt=1.5_dtslice=0.125
28,2,2,1.50,0.250,suzuki2,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=suzuki2_dt=1.5_dtslice=0.25
30,2,2,2.00,0.125,suzuki2,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=suzuki2_dt=2.0_dtslice=0.125


In [49]:
top5_df

,N,cutoff,dt,dt_slice_max,synthesis,final_k,final_reduction,reduction_at_first_idx,best_diff_to_min,first_idx_tol,max_num_2q_ops,num_2q_ops_at_first_idx,folder
0,2,2,1.00,0.125,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.125
1,2,2,1.00,0.250,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.25
2,2,2,1.00,0.500,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.5
3,2,2,1.50,0.125,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.5_dtslice=0.125
4,2,2,1.50,0.250,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.5_dtslice=0.25
5,2,4,0.25,0.500,suzuki2,3,56.250000,60.937500,1.895321e-16,0,6,6,synth=suzuki2_dt=0.25_dtslice=0.5
6,2,4,0.50,0.500,suzuki2,2,60.937500,60.937500,2.168133e-16,0,6,6,synth=suzuki2_dt=0.5_dtslice=0.5
7,2,4,3.00,0.125,lie,2,60.937500,62.500000,2.427664e-16,0,6,6,synth=lie_dt=3.0_dtslice=0.125
8,2,4,2.00,0.500,lie,3,57.812500,60.937500,2.446638e-16,0,6,6,synth=lie_dt=2.0_dtslice=0.5
9,2,4,3.00,0.500,lie,2,62.500000,62.500000,2.450161e-16,0,6,6,synth=lie_dt=3.0_dtslice=0.5


In [50]:
df = pd.concat(top_5s, ignore_index=True)
df[df['cutoff']==2]

,N,cutoff,dt,dt_slice_max,synthesis,final_k,final_reduction,reduction_at_first_idx,best_diff_to_min,first_idx_tol,max_num_2q_ops,num_2q_ops_at_first_idx,folder
0,2,2,1.0,0.125,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.125
1,2,2,1.0,0.250,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.25
2,2,2,1.0,0.500,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.0_dtslice=0.5
3,2,2,1.5,0.125,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.5_dtslice=0.125
4,2,2,1.5,0.250,lie,2,50.000000,50.000000,2.775558e-16,0,2,2,synth=lie_dt=1.5_dtslice=0.25
20,3,2,1.5,0.250,lie,2,62.500000,62.500000,0.000000e+00,0,54,30,synth=lie_dt=1.5_dtslice=0.25
21,3,2,1.0,0.500,lie,2,62.500000,64.062500,1.387779e-17,0,22,14,synth=lie_dt=1.0_dtslice=0.5
22,3,2,1.5,0.125,suzuki2,2,62.500000,62.500000,4.163336e-17,0,104,56,synth=suzuki2_dt=1.5_dtslice=0.125
23,3,2,3.0,0.250,suzuki2,2,62.500000,64.062500,4.163336e-17,0,104,56,synth=suzuki2_dt=3.0_dtslice=0.25
24,3,2,0.5,0.500,suzuki2,2,64.062500,67.187500,5.551115e-17,0,16,12,synth=suzuki2_dt=0.5_dtslice=0.5
